# Ground Truth Entity Recognition

In [1]:
# %pip install awswrangler
# %pip install bleach
# %pip install nltk

In [3]:
import os
import sys
import nltk
import json
import boto3 
import numpy as np
import pandas as pd
import awswrangler as wr
from bleach.sanitizer import Cleaner

nltk.download('punkt')

from sagemaker import get_execution_role
role = get_execution_role()

sys.path.append(os.path.dirname(os.getcwd()) + '/src')
print('added ', os.path.dirname(os.getcwd()) + '/src', ' to sys.')
from preprocess import preprocess_data

added  /home/ec2-user/SageMaker/ons-poc-manobras-catalog/src  to sys.


[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
preproc = preprocess_data()

Preprocessing module
started at  2021-09-15 20:21:17.850813


In [5]:
# reading file contents from s3 bucket
df = preproc.get_baseline_text()
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6025 entries, 0 to 6024
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Id                       6025 non-null   int64  
 1   documentUID              6025 non-null   object 
 2   contentUID               6025 non-null   object 
 3   Title                    6023 non-null   object 
 4   Level                    6025 non-null   object 
 5   hierarquia               6025 non-null   object 
 6   relatedEmbedded          0 non-null      float64
 7   relatedLinks             0 non-null      float64
 8   MpoAreaEletrica          6025 non-null   object 
 9   MpoCentro                6025 non-null   object 
 10  MpoEquipamentos          2370 non-null   object 
 11  MpoIdentificador         6025 non-null   object 
 12  MpoLigadoDesligado       6025 non-null   object 
 13  MpoLocalizacaoTaxonomia  6025 non-null   object 
 14  MpoMopsAssociadas       

,Id,documentUID,contentUID,Title,Level,hierarquia,relatedEmbedded,relatedLinks,MpoAreaEletrica,MpoCentro,...,MpoRevisaoReadOnly,MpoSubmodulo,MpoTema,MpoTextoVigenciaPorData,MpoTipoDocumento,nomeDocumento,statusItem,StatusDocumento,texto,Created
0,181505,e401f716-b9c8-44cf-a961-75dd62a2a8aa,cffb9a6b-ba1f-4756-82d4-cd88a5230036,OBJETIVO,1,OBJETIVO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,Estabelecer procedimentos para manobras de equ...,30/07/2021 14:43:17
1,181506,e401f716-b9c8-44cf-a961-75dd62a2a8aa,7b8fcaee-0dce-41f1-b1d9-3dacc9871960,CONSIDERAÇÕES GERAIS,2,CONSIDERAÇÕES GERAIS,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,2.1. Os procedimentos contidos nest...,30/07/2021 14:43:18
2,181507,e401f716-b9c8-44cf-a961-75dd62a2a8aa,5a57338c-7b88-466c-a769-51ce87e128b1,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,3,PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRANSMISSÃO,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,1. OBJETIVO2....................................,30/07/2021 14:43:18
3,181508,e401f716-b9c8-44cf-a961-75dd62a2a8aa,2d387a90-353d-4411-b263-d528fbba8e7c,LT 230 KV Tucuruí / Altamira,3.1,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.1.1. Desenergização da LT 230 kV Tucur...,30/07/2021 14:43:18
4,181509,e401f716-b9c8-44cf-a961-75dd62a2a8aa,0ba555c0-1b6a-4d16-8f76-8758b214042c,LT 230 KV Altamira / Xingu,3.2,3 - PREPARAÇÃO PARA MANOBRAS DE LINHAS DE TRAN...,NaN,NaN,2TR,COSR-NCO,...,27,5.12 - Preparação para Manobras na Área 230 kV...,Instruções da Região Norte - Área 230 kV Tramo...,2020-03-11 03:00:00,Preparação para Manobras,Preparação para Manobras na Área 230 kV Tramo ...,Aprovado,Vigente,3.2.1. Desenergização da LT 230 kV Altam...,30/07/2021 14:43:18


In [6]:
# number of valid (False) samples
# df['MpoEquipamentos'].isnull().value_counts()

In [8]:
df_sub = preproc.get_subsections(dataset=df, column='texto')
df_sub

,reference,subsection,subsection_txt,actions
0,0,NaN,Estabelecer procedimentos para manobras de equ...,None
1,1,NaN,2.1. Os procedimentos contidos nest...,"[2.1., Os procedimentos contidos nesta instruç..."
2,1,NaN,2.2. Constam nesta instrução de ope...,"[2.2., Constam nesta instrução de operação os ..."
3,1,NaN,2.3. O restabelecimento de linhas d...,"[2.3., O restabelecimento de linhas de transmi..."
4,1,NaN,2.4. O restabelecimento de transfor...,"[2.4., O restabelecimento de transformadores, ..."
...,...,...,...,...
71475,6021,NaN,3.6.2. Energização da LT 230 kV A...,"[3.6.2., Energização da LT 230 kV Atlântida ..."
71476,6021,NaN,3.7. LT 230 kV Atlântida 2 / Osór...,"[3.7., LT 230 kV Atlântida 2 / Osório 23.7.1..."
71477,6022,NaN,4.87.2.1. Energização do Transformador TF-2 23...,None
71478,6023,NaN,PassoCoordenaçãoControleComando e ExecuçãoProc...,None


In [9]:
n = 110

print('Original preprocessed dataframe: \n')
for sample in [n]:
    print('-'*20)
    print(df['texto'].iloc[sample])

# print('\n','#'*20, '\n')
    
# print('Preprocessed dataframe with subsections: \n')
# for sample in [n]:
#     df_sub_example = df_sub[df_sub['reference']==sample]
#     for row in range(len(df_sub_example)):
#         print(df_sub_example['subsection'].iloc[row])
#         print(df_sub_example['subsection_txt'].iloc[row])
#         print('-'*20)
        
print('\n','#'*20, '\n')
    
print('Preprocessed dataframe with subsections: \n')
for sample in [n]:
    df_sub_example = df_sub[df_sub['reference']==sample]
    for row in range(len(df_sub_example)):
        print(df_sub_example['actions'].iloc[row])
        print('-'*20)

Original preprocessed dataframe: 

--------------------
 1. OBJETivo2. Considerações gerais3. PREPARAÇÃO PARA MANOBRAS em linha de transmissão3.1.              LT 345 kV Furnas / Estreito (sistêmico)3.1.1.              Desenergização da LT 345 kV Furnas / Estreito3.1.2.              Energização da LT 345 kV Furnas / Estreito3.2.              LT 345 kV Furnas / Mascarenhas de Moraes3.2.1.              Desenergização da LT 345 kV Furnas / Mascarenhas de Moraes3.2.2.              Energização da LT 345 kV Furnas / Mascarenhas de Moraes3.3.              LT 345 kV Furnas / Poços de Caldas C1 ou C2 (sistêmico)3.3.1.              Desenergização da LT 345 kV Furnas / Poços de Caldas C1 ou C2.3.3.2.              Energização da LT 345 kV Furnas / Poços de Caldas C1 ou C23.4.              LT 345 kV Itumbiara / Porto Colômbia3.4.1.              Desenergização da LT 345 kV Itumbiara / Porto Colômbia3.4.2.              Energização da LT 345 kV Itumbiara / Porto Colômbia3.5.              LT 345 kV Jag

In [10]:
# save output file

# bucket = 'ons-dl-dev-landing'
# file = 'ons/mpo_ds_nlp/raw/mpo_subsections.csv'
# obj_uri = 's3://{}/{}'.format(bucket, file)
# df.to_csv(obj_uri)